### Write a Program to validate the withdrawal request and display

In [1]:
from abc import ABC, abstractmethod
import uuid

In [2]:
SUCCESS_MSG = "Transaction completed"
FAILED_MSG = "Transaction failed!"

In [ ]:
class BankAccount:
    def __init__(self, holder_name: str, phone_no: str, initial_balance: float, minimum_balance: float = 0.0):
        if initial_balance < 0:
            raise ValueError("Initial Balance/Deposit can't be negative")
        
        if not holder_name.strip():
            raise ValueError("Enter a proper holder name")
        
        if not phone_no.strip() or len(phone_no) < 10 or len(phone_no) > 10:
            raise ValueError("Enter a proper phone number")
        
        self.holder_name = holder_name
        self.phone_no = phone_no
        self.account_no = str(uuid.uuid4().int)[:12]
        self.minimum_balance = minimum_balance
        self.__balance = initial_balance
        self.cards = []

    # This is a READ-ONLY property (Preventing the Direct access of Private variables)
    @property        
    def balance(self):
        return self.__balance
    
    def check_balance(self):
        return f"Your balance is: {self.balance}"
    
    def account_info(self):
        acc = f"Account holder name: {self.holder_name}"
        acc_no = f"Account no: {self.account_no}"
        acc_balance = f"Account balance: {self.balance}"
        acc_min_bal = f"Minimum balance is: {self.minimum_balance}"
        return f"{acc}\n{acc_no}\n{acc_balance}\n{acc_min_bal}"
    
    # These methods are safely accessing the Private Variables
    # To ADD MONEY into MY BANK ACCOUNT
    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Transaction failed, amount can't be less than or equal to zero")
        
        self.__balance += amount
        return SUCCESS_MSG

    # TO TAKE MONEY from MY BANK ACCOUNT
    def withdrawal(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Amount can't be less than or equal to zero!")
        
        elif amount > self.__balance:
            raise ValueError("Withdrawal amount can't be greater than balance")
        
        elif self.__balance - amount < self.minimum_balance:
            raise ValueError("Minimum balance must be maintained")
        
        self.__balance -= amount
        return SUCCESS_MSG
    


In [4]:
# Abstract class which will be common for all source of payment gateways
class Payment(ABC):
    @abstractmethod
    def pay(self, account: BankAccount, pin: int, amount: float):
        pass

In [17]:
class Bank(ABC):
    @abstractmethod
    def open_savings_account(self):
        pass

    @abstractmethod
    def open_current_account(self):
        pass

    @abstractmethod
    def close_savings_account(self):
        pass

    @abstractmethod
    def close_current_account(self):
        pass

In [5]:
class Card(ABC):
    @abstractmethod
    def create_card(self, account: BankAccount):
        pass

    @abstractmethod
    def validate_pin(self, pin: str, confirm_pin: str):
        pass

    def generate_card_number(self):
        return str(uuid.uuid4().int)[:16]
    
    @abstractmethod
    def show_card(self):
        pass

In [6]:
class DebitCard(Card):
    def __init__(self, holder_name: str):
        if not holder_name.strip():
            raise ValueError("Enter a Valid Account holder name!")
        self.holder_name = holder_name
        self.card_no = 0
        self.__pin = 0
    
    # Creates a Pin for the DebitCard
    def validate_pin(self, pin: str, confirm_pin: str) -> int:
        if len(pin) != 4:
            raise ValueError("Invalid Pin!")

        if len(confirm_pin) != 4 or confirm_pin != pin:
            raise ValueError("Invalid Pin!")

        return int(pin)
    
    def show_card(self):
        if not self.card_no:
            raise ValueError("No Card is created")
        
        return f"Holder name: {self.holder_name}\nCard Number: {self.card_no}"

    def create_card(self, account: BankAccount, pin: str, confirm_pin: str):
        if not isinstance(account, BankAccount):
            raise ValueError("Invalid Bank Account!")
        
        self.card_no = super().generate_card_number()

        try:
            self.__pin = self.create_pin(pin, confirm_pin)
            self.show_card()
        except ValueError:
            raise

In [7]:
class Upi(Payment):
    def __init__(self, upi_id: str, pin: int) -> None:
        if not upi_id.strip():
            raise ValueError("Invalid UPI Id")
        
        if not isinstance(pin, int):
            raise ValueError("Pin should contain numbers only")
        
        if len(str(pin)) != 4 and len(str(pin)) != 6:
            raise ValueError("PIN must be 4 or 6 digits")

        self.upi_id = upi_id
        self.__pin = pin

    def verify_pin(self, pin):
        return self.__pin == pin

    def pay(self, account: BankAccount, pin: int, amount: float):
        if not isinstance(account, BankAccount):
            raise ValueError("Invalid Bank Account!")
        
        if not self.verify_pin(pin):
            raise ValueError("Invalid PIN")
        
        try:
            account.withdrawal(amount)
            return SUCCESS_MSG
        except ValueError:
            raise

In [16]:
print("""
Enter:
    1 -> To create Savings Account
    2 -> To create Current Account
    3 -> To Deposit Amount
    4 -> To Withdraw Amount
    5 -> To Create Debit Card
    6 -> To Activate Debit Card
    7 -> To Check Balance
    8 -> To View Bank Info
    9 -> To exit""")


Enter:
    1 -> To create Savings Account
    2 -> To create Current Account
    3 -> To Deposit Amount
    4 -> To Withdraw Amount
    5 -> To Create Debit Card
    6 -> To Activate Debit Card
    7 -> To Check Balance
    8 -> To View Bank Info
    9 -> To exit


In [13]:
def perform_transaction(account: BankAccount, choice: int, amount: float):
    try:
        if choice == 1:
            return account.deposit(amount)
        elif choice == 2:
            return account.withdrawal(amount)
    except ValueError as err:
        return err

In [ ]:
initial_balance = float(input("Enter a Initial Balance: "))
holder_name = input("Enter a Account holder name: ")
try:
    print(f"Account holder name: {holder_name}\nInitial Balance set: {initial_balance}\n")
    account = BankAccount(holder_name, initial_balance)

    while True: 
        choice = int(input("\nEnter your choice: "))
        print(f"\nEntered Choice is: {choice}")

        if choice == 1:
            amount = float(input("Enter the deposit amount: "))
            print(f"Amount to be deposited is: {amount}")
            
            response = perform_transaction(account, 1, amount)
            print(response)

        elif choice == 2:
            amount = float(input("Enter the withdrawal amount: "))
            print(f"Withdrawal amount is: {amount}")

            response = perform_transaction(account, 2, amount)
            print(response)
            
        elif choice == 3:
            # Create a Debit Card

        elif choice == 4:
            # withdraw using UPI

        elif choice == 5:
            minimum_balance = float(input("Enter the new Minimum Balance: "))
            print(f"New Minimum Balance is: {minimum_balance}")
            account.minimum_balance = minimum_balance
                
        elif choice == 6:
            response = account.check_balance()
            print(response)

        elif choice == 7:
            response = account.account_info()
            print(response)

        elif choice == 8:
            print("Bye!!")
            break
        
        else:
            print("Enter a valid input")

except ValueError as e:
    print(f"Error: {e}")

Account holder name: Sayan Mondal
Initial Balance set: 500.0


Entered Choice is: 1
Amount to be deposited is: 500.0
Transaction completed

Entered Choice is: 7
Account holder name: Sayan Mondal
Account no: 281969064391
Account balance: 1000.0
Minimum balance is: 0.0

Entered Choice is: 2
Withdrawal amount is: 10000.0
Withdrawal amount can't be greater than balance

Entered Choice is: 2
Withdrawal amount is: 1.0
Transaction completed

Entered Choice is: 7
Account holder name: Sayan Mondal
Account no: 281969064391
Account balance: 999.0
Minimum balance is: 0.0

Entered Choice is: 5
New Minimum Balance is: 500.0

Entered Choice is: 7
Account holder name: Sayan Mondal
Account no: 281969064391
Account balance: 999.0
Minimum balance is: 500.0

Entered Choice is: 6
Your balance is: 999.0

Entered Choice is: 1
Amount to be deposited is: 5000.0
Transaction completed

Entered Choice is: 7
Account holder name: Sayan Mondal
Account no: 281969064391
Account balance: 5999.0
Minimum balance is: 500.